In [54]:
import imaplib
import email
from email.header import decode_header
import os
import datetime
from dateutil import parser
import pandas as pd

In [71]:
def get_email_data(username, passsword, a_date, verbose):
    
    
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    imap.login(username, password)

    status, messages = imap.select("INBOX")

    before_date_flag = 0

    # total number of emails
    messages = int(messages[0])

    subject_list = []
    category_list = []
    date_list = []
    sender_list = []
    
    
    for i in range(messages,0,-1):

        if before_date_flag == 1:
            break
        res, msg = imap.fetch(str(i), "(RFC822)")
        for response in msg:
            if isinstance(response, tuple):
            
                # parse a bytes email into a message object
                msg = email.message_from_bytes(response[1])

                date = parser.parse(msg['Date']).date()
                subject = decode_header(msg["Subject"])[0][0]

                if date < a_date:
                    before_date_flag = 1
                    break


                if isinstance(subject, bytes):
                    # if it's a bytes, decode to str
                    subject = subject.decode()

                date = msg["Date"][0:25]
                from_ = msg.get("From")

                sender_list.append(from_)
                subject_list.append(subject)
                date_list.append(date)
                if 'Thank you for applying'.lower() in subject.lower():
                    category_list.append(1)
                else:
                    category_list.append(0)
                    
                if verbose:
                    print("Subject:", subject)
                    print("Date:", date)
                    print("="*100)

    # close the connection and logout
    imap.close()
    imap.logout()
    
    return {'Sender': sender_list,
            'Subject': subject_list,
            'Date': date_list,
            'Category': category_list,
           }

In [72]:
def num_to_str(x):
    if x == 1:
        return 'Job'
    else:
        return 'Other'

In [74]:
if __name__ == '__main__':
    mail_id = input('Enter mail id: ')
    mail_pswd = input('Enter mail password: ')
    app_date = input('Enter date of application (for the job): ')
    
    app_date = parser.parse(app_date).date()
    
    email_data = get_email_data(mail_id, mail_pswd, app_date, True)
    
    mail_df = pd.DataFrame.from_dict(email_data)
    mail_df['Category'] = mail_df['Category'].apply(num_to_str)
    
    print(mail_df)

Enter mail id: sarthaksash99@gmail.com
Enter mail password: hello4mtheotherside
Enter date of application (for the job): 10-10-2020
Subject: Thank you for applying
Date: Tue, 13 Oct 2020 12:19:32
Subject: Sarthak, 15 New Internships Matching Your Profile - Apply Today!
Date: Tue, 13 Oct 2020 12:17:23
Subject: New Color. Gunmetal.
Date: Tue, 13 Oct 2020 11:29:43
Subject: 💥News Flash: Exclusive Deals Await You
Date: Tue, 13 Oct 2020 10:45:40
Subject: Sarthak: 21 new jobs for 'data analyst' in Bengaluru, Karnataka,
 India
Date: Tue, 13 Oct 2020 10:09:42
Subject: Sarthak: 25 new jobs for 'ios developer' in Bengaluru, Karnataka,
 India
Date: Tue, 13 Oct 2020 10:09:42
Subject: Sarthak: 12 new jobs for 'user experience designer' in Bengaluru,
 Karnataka, India
Date: Tue, 13 Oct 2020 10:09:42
Subject: A friend wants you to like a Page on Facebook
Date: Tue, 13 Oct 2020 02:16:12
Subject: Critical security alert
Date: Tue, 13 Oct 2020 08:27:48
Subject: Reminder 2: Exam Registration is open now f

Subject: The Island That Humans Can’t Conquer
Date: Sat, 10 Oct 2020 17:09:57
Subject: Join us for a day of self-care and support!
Date: Sat, 10 Oct 2020 17:11:25
Subject: 🤩 Sarthak, tick off your bucket list with Easiest Internships Ever
Date: Sat, 10 Oct 2020 13:22:28
Subject: ⚠️New Product Alert!⚠️ Introducing our all-new Pasta Pizzas! 😋
Date: Sat, 10 Oct 2020 11:56:46
Subject: Sarthak: 9 new jobs for 'user experience designer' in Bengaluru,
 Karnataka, India
Date: Sat, 10 Oct 2020 09:54:16
Subject: Sarthak: 30+ new jobs for 'data analyst' in Bengaluru, Karnataka,
 India
Date: Sat, 10 Oct 2020 09:54:16
Subject: Sarthak: 8 new jobs for 'ios developer' in Bengaluru, Karnataka,
 India
Date: Sat, 10 Oct 2020 09:54:16
Subject: How often do you think about “YOU”?
Date: Sat, 10 Oct 2020 08:23:46
Subject: Reminder: Exam Registration is open now for July 2020 Courses -
 Register Today!
Date: Sat, 10 Oct 2020 10:39:06
Subject: New Horizon College of Engineering and others share their thoughts